### Setup

In [1]:
import requests
import csv

ssl_cert_file = "../ssl/cert.pem"
url = "https://192.168.1.100:8000/"

username = "test"
file_username = "81"

In [2]:
response = requests.get(url, verify=ssl_cert_file)
if response.status_code == 200:
    print("Response from server:", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server: {'message': 'Hello, World!'}


### Training
Remember to move file_username file from training directory (so it's not included as negative example).

For single file:

In [ ]:
tsv_file_path = f"../datasets/{file_username}.tsv"
num_stages = 5

with open(tsv_file_path, "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter="\t")
    headers = next(reader)
    rows = list(reader)


rows_per_stage = len(rows) // num_stages
for i in range(num_stages - 1):
    start_index = rows_per_stage * i
    end_index = start_index + rows_per_stage
    
    tsv_data = "\t".join(headers) + "\n"
    tsv_data += "\n".join(["\t".join(row) for row in rows[start_index:end_index]])
    
    response = requests.post(
        url + 'upload_tsv',
        params={"username": username},
        data=tsv_data,
        verify=ssl_cert_file,
    )

    if response.status_code == 200:
        print("Response from server:", response.json())
    else:
        print("Error:", response.status_code, response.text)

In [ ]:
tsv_data = "\t".join(headers) + "\n"
tsv_data += "\n".join(["\t".join(row) for row in rows[rows_per_stage * (num_stages - 1):]])

response = requests.post(
    url + 'train',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:", response.json())
else:
    print("Error:", response.status_code, response.text)

For multiple files:

In [3]:
tsv_file_base = f"../datasets/key_presses_"
num_stages = 5

for i in range(num_stages - 1):
    with open(f"{tsv_file_base}{file_username}.{i + 1}.tsv", "r", encoding="utf-8") as file:
        reader = csv.reader(file, delimiter="\t")
        headers = next(reader)
        rows = list(reader)
        
        tsv_data = "\t".join(headers) + "\n"
        tsv_data += "\n".join(["\t".join(row) for row in rows])
        
        response = requests.post(
            url + 'upload_tsv',
            params={"username": username},
            data=tsv_data,
            verify=ssl_cert_file,
        )
    
        if response.status_code == 200:
            print("Response from server:", response.json())
        else:
            print("Error:", response.status_code, response.text)

Response from server: {'message': 'TSV data received successfully'}
Response from server: {'message': 'TSV data received successfully'}
Response from server: {'message': 'TSV data received successfully'}
Response from server: {'message': 'TSV data received successfully'}


In [4]:
with open(f"{tsv_file_base}{file_username}.{num_stages}.tsv", "r", encoding="utf-8") as file:
    reader = csv.reader(file, delimiter="\t")
    headers = next(reader)
    rows = list(reader)
    
    tsv_data = "\t".join(headers) + "\n"
    tsv_data += "\n".join(["\t".join(row) for row in rows])
    
    response = requests.post(
        url + 'train',
        params={"username": username},
        data=tsv_data,
        verify=ssl_cert_file,
    )

    if response.status_code == 200:
        print("Response from server:", response.json())
    else:
        print("Error:", response.status_code, response.text)

Response from server: {'message': 'TSV data received successfully. Training succeeded.'}


### Inference

In [5]:
tsv_file_path = f"../datasets/key_presses_81.-1.tsv"

with open(tsv_file_path, "r", encoding="utf-8") as file:
    tsv_data = file.read()


response = requests.post(
    url + 'inference',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:\n", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server:
 {'message': 'TSV data received successfully. Inference succeeded', 'score': 0.1666666716337204, 'prediction': 0}


In [6]:
tsv_file_path = f"../datasets/key_presses_82.-1.tsv"

with open(tsv_file_path, "r", encoding="utf-8") as file:
    tsv_data = file.read()


response = requests.post(
    url + 'inference',
    params={"username": username},
    data=tsv_data,
    verify=ssl_cert_file,
)

if response.status_code == 200:
    print("Response from server:\n", response.json())
else:
    print("Error:", response.status_code, response.text)

Response from server:
 {'message': 'TSV data received successfully. Inference succeeded', 'score': 0.0, 'prediction': 0}
